In [1]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import scipy
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [2]:
import urllib.request as request

**Imports all traffic data from 2007 - 2018**

Link: https://data.seattle.gov/browse?q=traffic%20flow%20count&sortBy=relevance to download yearly data

In [3]:
url_list = ['https://data-seattlecitygis.opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.csv']

In [7]:
def get_df(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year - 7
    df_year = pd.read_csv(url_list[num])
    df_year_AAWDT = df_year['AAWDT'].values
    df_year_geobase = df_year['GEOBASEID'].values
    df_year_dist = df_year['SHAPE_Length'].values
    
    return df_year, df_year_AAWDT, df_year_geobase, df_year_dist

In [5]:
df_07 = get_df(7)

In [6]:
df_07.head()

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length
0,1,94250370,N,12522,RENTON AVE S,135,"RENTON AVE S, N/O S CLOVERDALE ST",2007,12522,7200,0,,Y,4615.134504
1,2,50000650,N,8702,AIRPORT WAY S,208,"AIRPORT WAY S, N/O S NORFOLK ST",2007,8702,13100,0,,Y,18205.478931
2,3,127150040,N,16114,N NORTHGATE WAY,56,"N NORTHGATE WAY, W/O ASHWORTH AVE N",2007,16114,28300,0,,Y,4267.857028
3,4,123450090,N,15794,N 65TH ST,127,"N 65TH ST, W/O LINDEN AVE N",2007,15794,8200,0,,Y,12341.641545
4,5,9550200,Y,2762,2ND AVE,145,"2ND AVE, NW/O LENORA ST",2007,2762,12400,0,,Y,4161.475805
